# Gapfilling NDVI/LST with machine learning derived synthetic datasets


### Load packages
Import Python packages that are used for the analysis.

In [ ]:
%matplotlib inline

import xarray as xr
import numpy as np
import pandas as pd
import seaborn as sb
from joblib import dump, load
from scipy import stats
import geopandas as gpd
from pprint import pprint
from odc.geo.xr import assign_crs
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

import lightgbm as lgbm
from lightgbm import LGBMRegressor

import shap
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import sys
sys.path.append('/g/data/os22/chad_tmp/dea-notebooks/Tools/')
from dea_tools.classification import predict_xr, HiddenPrints
from dea_tools.spatial import xr_rasterize

sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import round_coords

import warnings
warnings.filterwarnings("ignore")

# Using ggplot styles in this notebook
plt.style.use('ggplot')

%matplotlib inline

### Analysis parameters


In [ ]:
model_var='NDVI'
feat = 'CLIM'
n_samples = 30000
n_val = 5000
name = 'desert' #nondesert
base = '/g/data/os22/chad_tmp/AusENDVI/data/'

### Assemble datasets for training and predicting

In [ ]:
if model_var == 'NDVI':
    datasets = [
         f'{model_var}_harmonization/LGBM/{model_var}_{feat}_LGBM_5km_monthly_1982_2022_wGaps.nc',
        '5km/MOY_5km_monthly_1982_2022.nc',
        '5km/rain_5km_monthly_1981_2022.nc',
        '5km/rain_cml3_5km_monthly_1982_2022.nc',
        '5km/rain_cml6_5km_monthly_1982_2022.nc',
        '5km/rain_anom_5km_monthly_1981_2022.nc',
        '5km/rain_cml3_anom_5km_monthly_1982_2022.nc',
        '5km/rain_cml6_anom_5km_monthly_1982_2022.nc',
        '5km/vpd_5km_monthly_1982_2022.nc',
        '5km/srad_5km_monthly_1982_2022.nc',
        '5km/srad_anom_5km_monthly_1982_2022.nc',
        '5km/tavg_5km_monthly_1982_2022.nc',
        '5km/tavg_anom_5km_monthly_1982_2022.nc',
        # '5km/Elevation_5km_monthly_1982_2022.nc',
        '5km/CO2_5km_monthly_1982_2022.nc',
        '5km/WCF_5km_monthly_1982_2022.nc',
        # '5km/Aspect_5km_monthly_1982_2022.nc'
               ]

if model_var == 'LST':
    datasets = [
         f'{model_var}_harmonization/LGBM/{model_var}_{feat}_LGBM_5km_monthly_1982_2022_wGaps.nc',
        '5km/MOY_5km_monthly_1982_2022.nc',
        '5km/vpd_5km_monthly_1982_2022.nc',
        '5km/srad_5km_monthly_1982_2022.nc',
        '5km/srad_anom_5km_monthly_1982_2022.nc',
        '5km/tavg_5km_monthly_1982_2022.nc',
        '5km/tavg_anom_5km_monthly_1982_2022.nc',
        '5km/rain_5km_monthly_1981_2022.nc',
        '5km/rain_anom_5km_monthly_1981_2022.nc',
        # '5km/Elevation_5km_monthly_1982_2022.nc',
        '5km/WCF_5km_monthly_1982_2022.nc',
        # '5km/Aspect_5km_monthly_1982_2022.nc'
               ]

In [ ]:
dss = []
for d in datasets:
    xx = xr.open_dataset(base+d).sel(time=slice('1982','2022'))
    xx = assign_crs(xx, crs ='epsg:4326')
    xx = round_coords(xx)
    xx = xx.drop('spatial_ref')
    dss.append(xx)

ds = xr.merge(dss)
ds = assign_crs(ds, crs ='epsg:4326')
# ds = ds.sel(time=slice('2001','2022'))

### Add modis summary stats

In [ ]:
mean_modis = ds[model_var].median('time')
mean_modis = mean_modis.expand_dims(time=ds.time)
ds[model_var+'_50'] = mean_modis

min_modis = ds[model_var].quantile(0.05, dim='time').drop('quantile')
min_modis = min_modis.expand_dims(time=ds.time)
ds[model_var+'_05'] = min_modis

max_modis = ds[model_var].quantile(0.95, dim='time').drop('quantile')
max_modis = max_modis.expand_dims(time=ds.time)
ds[model_var+'_95'] = max_modis

### Add derivative of rainfall

In [ ]:
# if model_var == 'NDVI':
#     derived = ds['rain'].differentiate(coord='time', datetime_unit='D')
#     ds['rain_gradient'] = derived

### mask for desert/nondesert

In [ ]:
gdf = gpd.read_file('/g/data/os22/chad_tmp/AusENDVI/data/bioclimatic_regions.geojson')

desert = gdf[5:6] # grab desert
mask = xr_rasterize(desert, ds)
mask = round_coords(mask)

if name=='nondesert':
    mask = ~mask

mask.plot(size=3);

In [ ]:
ds = ds.where(mask)

## Training & testing data: equal random sampling of bioclimatic regions

Can skip to importing TD if already run this

In [ ]:
# Dictionary to save results 
results = []
for index, row in gdf.iterrows():
    
    if name=='desert':
        n=int(n_samples/1)
        if (row['region_name']!='Desert'):
            continue

    if name=='nondesert':
        n = int(n_samples/5)

    if (name=='nondesert') & (row['region_name']=='Desert'):
        continue
    
    print(row['region_name'], n)

    # Generate a polygon mask to keep only data within the polygon
    mask = xr_rasterize(gdf.iloc[[index]], ds[model_var])
    mask = round_coords(mask)
    
    # Mask dataset to set pixels outside the polygon to `NaN`
    dss = ds.where(mask)

    #sample equivalent num of samples per region
    df = dss.to_dataframe().dropna().sample(n=n, random_state=0).reset_index()
    
    # Append results to a dictionary using the attribute
    # column as an key
    results.append(df)


In [ ]:
df = pd.concat(results).reset_index(drop=True)
df['year'] = pd.DatetimeIndex(df['time']).year

In [ ]:
df = df.drop(['time','spatial_ref'], axis=1)

### Independent validation samples

In [ ]:
validation = df.sample(n=n_val, random_state=0)

In [ ]:
df = df.drop(validation.index)
print(len(df), 'training samples')

### Plot the location of the samples

In [ ]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326"
)

gdf_val = gpd.GeoDataFrame(
    validation, geometry=gpd.points_from_xy(validation.longitude, validation.latitude), crs="EPSG:4326"
)

gdf = gdf.drop(['longitude'], axis=1)
gdf_val = gdf_val.drop(['longitude'], axis=1)

In [ ]:
gdf_val.plot(column='year', cmap='inferno', markersize=1)

In [ ]:
gdf.to_file(f'{base}synthetic/{model_var}/{model_var}_{name}_training_data.geojson')
gdf_val.to_file(f'{base}synthetic/{model_var}/{model_var}_{name}_validation_data.geojson')

### Import training data if skipping above

In [ ]:
# gdf = gpd.read_file(f'{base}synthetic/{model_var}/{model_var}_{name}_training_data.geojson')
# gdf_val = gpd.read_file(f'{base}synthetic/{model_var}/{model_var}_{name}_training_data.geojson')

In [ ]:
df = pd.DataFrame(gdf.drop(columns='geometry', axis=1))
validation = pd.DataFrame(gdf_val.drop(columns='geometry', axis=1))

df = df.drop(['year'], axis=1)
validation = validation.drop(['year'], axis=1)

y = df[model_var]
x = df.drop([model_var], axis=1)

## Testing model using nested CV

In [ ]:
# Create the parameter grid using distributions
param_grid = {
    'num_leaves': stats.randint(5,50),
    'min_child_samples':stats.randint(10,30),
    'boosting_type': ['gbdt', 'dart'],
    'max_depth': stats.randint(5,25),
    'n_estimators': [300, 400, 500],
}

In [ ]:
outer_cv = KFold(n_splits=5, shuffle=True,
                   random_state=0)

# lists to store results of CV testing
acc = []
rmse=[]
r2=[]

i = 1
for train_index, test_index in outer_cv.split(x, y):
    print(f"Working on {i}/5 outer CV split", end='\r')
    model = LGBMRegressor(random_state=1,
                          verbose=-1,
                          # n_jobs=-1
                          )

    # index training, testing
    X_tr, X_tt = x.iloc[train_index, :], x.iloc[test_index, :]
    y_tr, y_tt = y.iloc[train_index], y.iloc[test_index]
    
    #simple random split on inner fold
    inner_cv = KFold(n_splits=3,
                     shuffle=True,
                     random_state=0)
    
    clf = RandomizedSearchCV(
                   model,
                   param_grid,
                   verbose=0,
                   n_iter=50,
                   # n_jobs=-1,
                   cv=inner_cv.split(X_tr, y_tr)
                  )
    
    #prevents extensive print statements
    clf.fit(X_tr, y_tr, callbacks=None)
    
    # predict using the best model
    best_model = clf.best_estimator_
    pred = best_model.predict(X_tt)

    # evaluate model w/ multiple metrics
    # r2
    r2_ = r2_score(y_tt, pred)
    r2.append(r2_)
    # MAE
    ac = mean_absolute_error(y_tt, pred)
    acc.append(ac)
    # RMSE
    rmse_ = np.sqrt(mean_squared_error(y_tt, pred))
    rmse.append(rmse_)
    
    #1:1 plots for each fold (save to csv so we can make a plot later on)
    df = pd.DataFrame({'Test':y_tt, 'Pred':pred}).reset_index(drop=True)

    df.to_csv(f'{base}synthetic/{model_var}/cross_val/{i}_{model_var}_{name}.csv')
    
    i += 1

### Create a single 1:1 plot out of the folds 


In [ ]:
dffs=[]
for i in range(1,5+1):
    df = pd.read_csv(f'{base}synthetic/{model_var}/cross_val/{i}_{model_var}_{name}.csv',
                     usecols=['Test', 'Pred'])
    dffs.append(df)

cross_df = pd.concat(dffs)

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(6,6))

xy = np.vstack([cross_df['Test'],cross_df['Pred']])
z = gaussian_kde(xy)(xy)

sb.scatterplot(data=cross_df, x='Test',y='Pred',cmap='magma', c=z, s=50, lw=1, alpha=0.5, ax=ax)
sb.regplot(data=cross_df, x='Test',y='Pred', scatter=False, color='darkblue', ax=ax)
sb.regplot(data=cross_df, x='Test',y='Test', color='black', scatter=False, line_kws={'linestyle':'dashed'}, ax=ax);

plt.xlabel('Observation '+model_var, fontsize=16)
plt.ylabel('Prediction '+model_var, fontsize=16)
ax.text(.05, .95, 'r\N{SUPERSCRIPT TWO}={:.2f}'.format(np.mean(r2)),
            transform=ax.transAxes, fontsize=16)
ax.text(.05, .9, 'MAE={:.2g}'.format(np.mean(acc)),
            transform=ax.transAxes, fontsize=16)

if model_var=='NDVI':
    ax.set_ylim(0, 1)
    ax.set_xlim(0, 1)

ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)

plt.tight_layout()
fig.savefig(f'{base}synthetic/{model_var}/cross_val/cross_val_{model_var}_{name}.png',
            bbox_inches='tight', dpi=300)

## Optimize model using all training data

Using a randomized strategy so we can search through more variables, with 500 iterations


In [ ]:
outer_cv = KFold(n_splits=5, shuffle=True,
                   random_state=0)

clf = RandomizedSearchCV(LGBMRegressor(verbose=-1),
                   param_grid,
                   verbose=1,
                   n_iter=250,
                   # n_jobs=-1,
                   cv=outer_cv
                  )

clf.fit(x, y, callbacks=None)

In [ ]:
print("The most accurate combination of tested parameters is: ")
pprint(clf.best_params_)
print('\n')
print("The best score using these parameters is: ")
print(round(clf.best_score_, 2))

## Fit on all data using best params

In [ ]:
model = LGBMRegressor(**clf.best_params_)

model.fit(x,y)

## Compare with independent validation data

In [ ]:
y_val = validation[model_var]
x_val = validation.drop([model_var], axis=1)

In [ ]:
pred = model.predict(x_val)

r2 = r2_score(y_val, pred)
ac = mean_absolute_error(y_val, pred)
df_val = pd.DataFrame({'Test':y_val, 'Pred':pred}).reset_index(drop=True)

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(6,6))

xy = np.vstack([df_val['Test'],df_val['Pred']])
z = gaussian_kde(xy)(xy)

sb.scatterplot(data=df_val, x='Test',y='Pred',cmap='magma',c=z, s=50, lw=1, alpha=0.5, ax=ax)
sb.regplot(data=df_val, x='Test',y='Pred', scatter=False, color='darkblue', ax=ax)
sb.regplot(data=df_val, x='Test',y='Test', color='black', scatter=False, line_kws={'linestyle':'dashed'}, ax=ax);

plt.xlabel('Observation '+model_var, fontsize=16)
plt.ylabel('Prediction '+model_var, fontsize=16)
ax.text(.05, .95, 'r\N{SUPERSCRIPT TWO}={:.2f}'.format(r2),
            transform=ax.transAxes, fontsize=16)
ax.text(.05, .9, 'MAE={:.2g}'.format(ac),
            transform=ax.transAxes, fontsize=16)

if model_var=='NDVI':
    ax.set_ylim(0, 1)
    ax.set_xlim(0, 1)

ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)

plt.tight_layout()
fig.savefig(f'{base}synthetic/{model_var}/cross_val/validation_{model_var}_{name}.png',
            bbox_inches='tight', dpi=300)

### Save the model

In [ ]:
dump(model, '/g/data/os22/chad_tmp/AusENDVI/results/models/gapfill/gapfill_'+model_var+'_'+name+'_LGBM.joblib')

In [ ]:
# model = load('/g/data/os22/chad_tmp/climate-carbon-interactions/results/models/gapfill/gapfill_'+model_var+'_'+name+'_LGBM.joblib')

## Examine feature importance using SHAP


In [ ]:
# explain the model's predictions using SHAP
explainer = shap.Explainer(model)
shap_values = explainer(x)

In [ ]:
vals= np.abs(shap_values.values).mean(0)
feature_importance = pd.DataFrame(list(zip(x.columns, vals)), columns=['col_name','feature_importance_vals'])
feature_importance.sort_values(by=['feature_importance_vals'],ascending=False,inplace=True)
feature_importance['col_name'] = feature_importance['col_name'].str.removesuffix("_RS")

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,7))
shap.plots.violin(shap_values, max_display=10, show=False, plot_type="layered_violin")
ax = plt.gca() 
plt.gcf().axes[-1].set_aspect('auto')
plt.gcf().axes[-1].set_box_aspect(15)
# plt.gcf().axes[-1].set_box_aspect(100)
ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)
if name == 'desert':
    ax.set_xlabel(model_var+ ' "Desert" '+'SHAP Value', fontsize=16)
    ax.set_xlim(-0.05, 0.05)

if name == 'nondesert':
    ax.set_xlabel(model_var+ ' "NonDesert" '+'SHAP Value', fontsize=16)
    ax.set_xlim(-0.1, 0.1)

fig.savefig(f'{base}synthetic/{model_var}/cross_val/feature_importance_{model_var}_{name}.png',
            bbox_inches='tight', dpi=300)

## Make gridded predictions

In [ ]:
from datacube.utils.dask import start_local_dask

client = start_local_dask(mem_safety_margin='2Gb')
client

### Load model

In [ ]:
# model = load('/g/data/os22/chad_tmp/climate-carbon-interactions/results/models/gapfill/gapfill_'+model_var+'_LGBM.joblib').set_params(n_jobs=1)

### Add lat as a variable

Plus ensure order of the variables is correct for predictions

In [ ]:
lat = ds.latitude
lat = lat.expand_dims(time=ds.time, longitude=ds.longitude)
lat = lat.transpose('time', 'latitude', 'longitude')
ds['latitude_data'] = lat

# lon = ds.longitude
# lon = lon.expand_dims(time=ds.time, latitude=ds.latitude)
# lon = lon.transpose('time', 'latitude', 'longitude')
# ds['longitude_gridded'] = lon

In [ ]:
ds['NDVI_50'].mean(['x','y']).plot()

In [ ]:
list(ds.data_vars)[1:-1]

In [ ]:
columns = list(ds.data_vars)[1:-1]
# columns.insert(0, 'longitude_gridded')
columns.insert(0, 'latitude_data')
ds = ds[columns]
ds = ds.rename({'latitude':'y', 'longitude':'x'})

In [ ]:
ds

### Re-create mask

In [ ]:
gdf = gpd.read_file('/g/data/os22/chad_tmp/AusENDVI/data/bioclimatic_regions.geojson')

desert = gdf[5:6] # grab desert
mask = xr_rasterize(desert, ds)
mask = round_coords(mask)

if name=='nondesert':
    mask = (~mask.astype(bool))

# also create a land mask
land_mask = ~np.isnan(ds.WCF.mean('time'))

mask.plot(size=3);

### Predict

In [ ]:
results = []
i=0
for i in range(0, len(ds.time)):
    print(" {:03}/{:03}\r".format(i + 1, len(range(0, len(ds.time)))), end="")
    with HiddenPrints():
        predicted = predict_xr(model,
                            ds.isel(time=i).drop('time'),
                            proba=False,
                            clean=True,
                            chunk_size=100000,
                              ).compute()
    
    # predicted = predicted.Predictions.where(~mask.isel(time=i))
    predicted = predicted.assign_coords(time=ds.isel(time=i).time).expand_dims(time=1)
    results.append(predicted.astype('float32'))
    i+=1

In [ ]:
yy = xr.concat(results, dim='time').sortby('time').rename({'Predictions':model_var})#.astype('float32')
yy = yy.where(land_mask)
yy = yy.where(mask.rename({'latitude':'y', 'longitude':'x'}))
yy = yy.rename({'y':'latitude', 'x':'longitude'})

In [ ]:
yy[model_var].mean(['longitude','latitude']).plot(figsize=(11,4))

## Export

In [ ]:
yy.to_netcdf(f'{base}synthetic/{model_var}/{model_var}_{feat}_{name}_synthetic_5km_monthly_1982_2022.nc')